# Epidemiologické modely

*KMA/VPM2*

*Jan Půlpán*

## Úvod

- úvod, základní popis šuplíkových modelů
- popis základního SIR - obecný (na co se používá a jak, nejen epidemiologie)
- různá rozšíření základního modelu

- Existuje spousta variací SIR modelů, ukažme si v krátkosti některé z nich:
    
    - konstatní populace vs. vitální dynamika (zahrnuje narození a úmrtí populace)
    - SIS (po recovery nenásleduje imunita)
    - SIRS (po recovery je jen krátká imunita) - tímhle bych se chtěl zabývat-- SEIS/SEIR (existuje latetní období, kdy nakažená osoba není infekční)
    - MSIR (děti se mohou rodit s imunitou)


- obecně tyto deterministrické modely pomocí ODE jdou použít v případě, že zahrnují velkou populaci (v matematickém měřítku snad i limitně nekonečnou, viz konec článku z wikipedie)


Zabýváme se "přihrádkovými" modely pro epidemiologii, kdy se populace dělí do jednotlivých přihrádek (=kategorií). 

Nejzákladnější model s kterým přišli v roce 1927 Kermack a McKendrick [citace] se označuje SIR. Jednotlivé písmena označují kategorie, S z anglického **S**usceptible (náchylný, myšleno k nákaze), I jako **I**nfectious (infekční) a R jako **R**ecovered (zotavený) někdy také Removed.

## SIR model


* přidat obrázek se šuplíkama (SIR)

\begin{align*}
S' &= - \beta I S,\\
I' &= \beta I S - \gamma I,\\
R' &= \gamma I\\
\end{align*}


Vysvětli, proč to dělím N (???)

\begin{align*}
S' &= - \frac{\beta I S}{N},\\
I' &= \frac{\beta I S}{N} - \gamma I,\\
R' &= \gamma I\\
\end{align*}

- popis soustavy SIR, co a jak znamená
- jaké jsou steady state solution (nulový klidový stav, endemický stabilní state)
- $R_0$ číslo
- ukazát jednoduchou numeriku, SIR graf + Stackplot


- soustava je nelineární
- existuje analytické řešení
    - dá se redukovat, protože $S(t) + I(t) + R(t) = konst. = N$
    - vypsat jak vypadá analytické řešení
- základní reprodukční číslo $R_0 = \frac{\beta}{\gamma}$
- steady-state solutions


In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.integrate import ode, solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import Markdown, display

plt.style.use("ggplot")

## Základní SIR model

Model rozděluje populaci do tří přihrádek (compartments):
- **S** z anglického Susceptible - náchylný, myšleno k nákaze,
- **I** jako Infectious - infekční, 
- **R** jako Removed - odstraněný, myšleno již neinfekční.

Celkovou populaci značíme $N$ a pro základní SIR model, kdy zanedbáváme běžné přírustky a úbytky populace mimo epidemii, pro ní v jakémkoliv čase platí $N=S(t)+I(t)+R(t)$.

Jednotlivé přihrádky jsou svázány pomocí soustavy 3 direfenciálních rovnic.

\begin{align*}
\frac{dS}{dt} &= - \frac{\beta I S}{N},\\
\frac{dI}{dt} &= \frac{\beta I S}{N} - \gamma I,\\
\frac{dR}{dt} &= \gamma I\\
\end{align*}

Kde $\beta$ je koeficient tempa růstu nakažených. V našem případě, kdy vztah udávající přírůstek infekčních dělíme velikostí populace $N$, udává průměrné množství osobních kontaktů za jednotku času (obvykle jeden den). Koeficient tempa zotavení $\gamma$ pak odpovídá průměrnému počtu dní, kdy je jeden člen populace infekční (značíme $D$). Platí pak $\gamma = \frac{1}{D}$. 

*Popsat $R_0$*

In [64]:
N = 1000        # velikost populace
R0 = 0          # počáteční podmínka pro R
I0 = 10         # počáteční podmínka pro I
S0 = N-I0-R0    # počáteční podmínka pro S
max_time = 400  # maximální ča, pro který model počítat

# definice modelu
def sir_ode(times,ivs,params):
    beta, gamma = params
    S,I,R = ivs
    N = S+I+R
    # ODEs
    dS = -beta*S*I/N
    dI = beta*S*I/N-gamma*I
    dR = gamma*I
    return [dS,dI,dR]


def sir_solve(beta, gamma):
    fig = plt.figure(figsize=(35,12))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    params = [beta, gamma]
    ivs = [S0,I0,R0]
    time = np.linspace(0,max_time,801)

    sir_sol = solve_ivp(fun=lambda t, y: sir_ode(t, y, params), 
                    t_span=[min(time),max(time)], 
                    y0=ivs, 
                    t_eval=time)

    t_sol = sir_sol['t']
    s_sol = sir_sol['y'][0]
    i_sol = sir_sol['y'][1]
    r_sol = sir_sol['y'][2]


    splot = ax1.plot(t_sol, s_sol, color='blue', linewidth=2, label='S')
    iplot = ax1.plot(t_sol, i_sol, color='red', linewidth=2, label='I')
    rplot = ax1.plot(t_sol, r_sol, color='green', linewidth=2, label='R')
    
    display(Markdown(r'$R_O$ = '+str(beta/gamma)))
    display(Markdown(r'$D$ = '+str(1/gamma)))
    
    plt.xlabel("t",fontweight="bold")
    legend = ax1.legend(loc=5,bbox_to_anchor=(1.1,0.5))
    frame = legend.get_frame()
    frame.set_facecolor("white")

    pal = ["#AA0000", "#00AA00", "#0000AA"]
    ax2.stackplot(t_sol,i_sol,r_sol,s_sol,colors=pal, alpha=0.5)
    plt.show()
    
interactive_plot = interactive(sir_solve, 
                               beta = widgets.FloatSlider(min=0, max=1, step=.05, value=0.1), 
                               gamma = widgets.FloatSlider(min=.01, max=.3, step=.01, value=0.05))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0, step=0.05), FloatSlider(value=0.05, …

## SIRS model s vitální dynamikou

Upravíme náš SIR model tak, že získaná imunita (přihrádka R) není na vždy, ale někteří již neinfekční a odstranění jedinci se znovu stávají náchylnými. Takovýto model se nazývá SIRS. Parametr $\nu$

Jednoduchý SIR model neuvažuje natalitu $\Lambda$ (počet nově narozených za rok) a mortalitu $\mu$ (podíl zemřelých za rok) protože je epidemie většinou dostatečně rychlá na to, aby dynamika těchto procesů mohla být zanedbána. My oba parametry teď ale zahrneme do SIRS modelu, mortalitu navíc nastavíme zvlášť pro jednotlivé S,I a R přihrádky. Zachováme ale konstatní velikost populace $N$, pomocí nastavení $\Lambda = \mu_1 S + \mu_2 I + \mu_3 R$.

Náš SIRS model vydadá nyní takto:

\begin{align*}
S' &= \Lambda - \frac{\beta I S}{N} - \mu_1 S + \nu R,\\
I' &= \frac{\beta I S}{N} - \mu_2 I - \gamma I,\\
R' &= \gamma I - (\mu_3 + \nu) R.\\
\end{align*}


Protože máme konstatní populaci a natalita kompenzuje mortalitu, rovnice můžeme upravit do tvaru

\begin{align*}
S' &= - \frac{\beta I S}{N} + \mu_2 I + (\mu_3 +\nu) R,\\
I' &= \frac{\beta I S}{N} - (\mu_2 + \gamma) I,\\
R' &= \gamma I - (\mu_3 + \nu) R.\\
\end{align*}




In [94]:
N = 1000        # velikost populace
R0 = 0          # počáteční podmínka pro R
I0 = 10         # počáteční podmínka pro I
S0 = N-I0-R0    # počáteční podmínka pro S
max_time = 400  # maximální ča, pro který model počítat

def sirs_ode(t,ivs,params):
    beta, gamma, mu2, mu3, nu = params
    S,I,R = ivs
    N = S+I+R
    # ODEs
    dS = -beta*S*I/N + mu2*I + (mu3+nu)*R
    dI = beta*S*I/N-(mu2+gamma)*I
    dR = gamma*I-(mu3+nu)*R
    return [dS,dI,dR]

def sirs_solve(beta, gamma, mu_2, mu_3, nu):
    fig = plt.figure(figsize=(35,12))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    
    params = [beta, gamma, mu_2, mu_3, nu]
    ivs = [S0,I0,R0]
    time = np.linspace(0,max_time,801)

    sirs_sol = solve_ivp(fun=lambda t, y: sirs_ode(t, y, params), 
                    t_span=[min(time),max(time)], 
                    y0=ivs, 
                    t_eval=time)

    t_sol = sirs_sol['t']
    s_sol = sirs_sol['y'][0]
    i_sol = sirs_sol['y'][1]
    r_sol = sirs_sol['y'][2]


    splot = ax1.plot(t_sol, s_sol, color='blue', linewidth=2, label='S')
    iplot = ax1.plot(t_sol, i_sol, color='red', linewidth=2, label='I')
    rplot = ax1.plot(t_sol, r_sol, color='green', linewidth=2, label='R')
    
    display(Markdown(r'$R_O$ = '+str(beta/(mu_2 +gamma)))) 
    display(Markdown(r'$D_i$ = '+str(1/gamma)))
    display(Markdown(r'$D_r$ = '+str(1/nu)))

    plt.xlabel("t",fontweight="bold")
    legend = ax1.legend(loc=5,bbox_to_anchor=(1.1,0.5))
    frame = legend.get_frame()
    frame.set_facecolor("white")

    pal = ["#AA0000", "#00AA00", "#0000AA"]
    ax2.stackplot(t_sol,i_sol,r_sol,s_sol,colors=pal, alpha=0.5)
    plt.show()
    
interactive_plot = interactive(sirs_solve, 
                               beta = widgets.FloatSlider(min=0, max=1, step=.05, value=0.1), 
                               gamma = widgets.FloatSlider(min=.01, max=.3, step=.01, value=0.05),
                               mu_2 = widgets.FloatSlider(min=0, max=.02, step=.005, value=0.01, readout_format='.3f'),
                               mu_3 = widgets.FloatSlider(min=0, max=0.01, step=0.002, value=0.01, readout_format='.3f'), # najít typickou mortalitu
                               nu = widgets.FloatLogSlider(min=-3, 
                                                           max=0, 
                                                           base=10, 
                                                           step=.02, 
                                                           value=0.01, 
                                                           description='nu')) # zjistit, co přesně znamená nu, a jakou má hodnotu
output = interactive_plot.children[-1]

interactive_plot

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0, step=0.05), FloatSlider(value=0.05, …

**Závěry:**
- pokud je $\nu=0$ (máme tedy SIR model) tak se projeví mortalita ... ???
- protože je mortalita kompenzovaná natalitou, tak se mi vůbec neprojevuje, nebo jen v extrémně vysokých případech mortality
- předchozí neni pravda, projevuje se to, pokud je nu nízké (imunita se ztrácí za dlouho)


- díky tomu, že se vrací R ––> S, vzniká mi endemický stacionární stav

- měnit mortalitu na I nedává smysl, protožej je kompenzovaná natalitou a to neodpovídá skutečnosti

In [45]:
np.round(0.01/365, 8)

2.74e-05

# SIRS s časově závislým $\beta$

In [10]:
def sirs_bt_ode(t,init,parms):
    b, period, g, m2, m3 = parms
    S,I,R = init
    N = S+I+R
    beta = b*(1+np.cos(np.pi*2/period*t))+b
    
    dS = -beta*S*I/N + m2*I + m3*R
    dI = beta*S*I/N-m2*I-g*I
    dR = g*I-m3*R
    
    return [dS,dI,dR]

In [16]:
population = 1000
i0 = 20
max_time = 1000

natality = 0.001
mortality = 0.001

def sirs_bt_solve(b, period, g,):
    
    
    #fig = plt.figure(figsize=(16,12))
    fig = plt.figure(figsize=(35,18))
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    
    parms = [b, period, g, natality, mortality]
    init = [population,i0,0]
    time = np.linspace(0,max_time,801)

    sirs_bt_sol = solve_ivp(fun=lambda t, y: sirs_bt_ode(t, y, parms), 
                    t_span=[min(time),max(time)], 
                    y0=init, 
                    t_eval=time)

    t_sol = sirs_bt_sol['t']
    s_sol = sirs_bt_sol['y'][0]
    i_sol = sirs_bt_sol['y'][1]
    r_sol = sirs_bt_sol['y'][2]


    splot = ax1.plot(t_sol, s_sol, color='blue', linewidth=2, label='S')
    iplot = ax1.plot(t_sol, i_sol, color='red', linewidth=2, label='I')
    rplot = ax1.plot(t_sol, r_sol, color='green', linewidth=2, label='R')

    plt.xlabel("t",fontweight="bold")
    #plt.ylabel("Number",fontweight="bold")
    legend = ax1.legend(loc=5,bbox_to_anchor=(1.1,0.5))
    frame = legend.get_frame()
    frame.set_facecolor("white")
    frame.set_linewidth(0)
#    plt.show()

    pal = ["#AA0000", "#00AA00", "#0000AA"]
    #plt.stackplot(sir_sol['t'],sir_sol['y'][1],sir_sol['y'][2],sir_sol['y'][0],labels=labels,colors=pal, alpha=0.5 )
    ax2.stackplot(t_sol,i_sol,r_sol,s_sol,colors=pal, alpha=0.5)
    
    ax3.plot(t_sol, b*(1+np.cos(np.pi*2/period*time))+b, color='black')
    ax3.set_ylim(0,1.55)
    
    plt.show()
    
interactive_plot = interactive(sirs_bt_solve, 
                               b=widgets.FloatSlider(min=0, max=.5, step=.05, value=0.1), 
                               period=widgets.IntSlider(min=10, max=1000, step=20, value=300), 
                               g=widgets.FloatSlider(min=0, max=.1, step=.01, value=0.05))
output = interactive_plot.children[-1]
output.layout.height = '1050px'
interactive_plot

interactive(children=(FloatSlider(value=0.1, description='b', max=0.5, step=0.05), IntSlider(value=300, descri…

Ještě přidat vakcinaci: http://hplgit.github.io/prog4comp/doc/pub/._p4c-solarized-Python021.html

Zdroje:
- Wikipedia: https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model
- 